# Project - AI Gold Investment Estimations Assistant
We'll create an AI Investment assistant focused on Gold prices with multiple agents:
- Gold price API integration
- Investment advice and timing recommendations
- Fake gold buying simulation
- Arabic translation agent
- Speech-to-text functionality


In [ ]:
# imports

import os
import json
import requests
import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from pydub import AudioSegment
from pydub.playback import play


In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Free API key for gold prices - sign up at https://metalpriceapi.com/
GOLD_API_KEY = os.getenv('METAL_PRICE_API_KEY', 'demo')  # Use 'demo' for testing
GOLD_API_URL = "https://api.metalpriceapi.com/v1/latest"

if GOLD_API_KEY:
    print(f"GOLD_API_KEY exists and begins {GOLD_API_KEY[:8]}")
else:
    print("GOLD_API_KEY not set")


In [ ]:
system_message = "You are a helpful AI Investment assistant specializing in Gold investments. "
system_message += "Give informative, courteous answers about gold prices and investment advice. "
system_message += "Always be accurate. If you don't know the answer, say so. "
system_message += "Provide investment timing recommendations based on current market conditions."


In [ ]:
# Gold Price Functions

def get_gold_price(country="USD"):
    """Get current gold price for a specific country/currency"""
    print(f"Tool get_gold_price called for {country}")
    
    # Currency mapping for different countries
    currency_map = {
        'usa': 'USD', 'united states': 'USD', 'us': 'USD',
        'uk': 'GBP', 'britain': 'GBP', 'england': 'GBP',
        'europe': 'EUR', 'germany': 'EUR', 'france': 'EUR',
        'japan': 'JPY', 'canada': 'CAD', 'australia': 'AUD',
        'india': 'INR', 'china': 'CNY', 'saudi arabia': 'SAR',
        'uae': 'AED', 'egypt': 'EGP'
    }
    
    currency = currency_map.get(country.lower(), country.upper())
    
    # Demo prices (realistic current gold prices as fallback)
    demo_prices = {
        'USD': 2350.50, 'GBP': 1890.25, 'EUR': 2180.75, 'JPY': 345000.00,
        'CAD': 3200.80, 'AUD': 3580.90, 'INR': 195000.50, 'CNY': 17200.25,
        'SAR': 8800.75, 'AED': 8650.30, 'EGP': 115000.80
    }
    
    price_per_ounce = None
    api_success = False
    
    try:
        # API call to get gold price
        params = {
            'api_key': GOLD_API_KEY,
            'base': 'XAU',  # Gold symbol
            'currencies': currency
        }
        
        response = requests.get(GOLD_API_URL, params=params, timeout=10)
        print(f"API Response Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"API Response Data: {data}")
            
            if 'rates' in data and currency in data['rates']:
                rate = data['rates'][currency]
                if rate > 0:  # Ensure valid rate
                    price_per_ounce = round(rate, 2)  # Rate is already price per ounce
                    api_success = True
                    print(f"Successfully got price from API: {price_per_ounce}")
                else:
                    print(f"Invalid rate from API: {rate}")
            else:
                print(f"Currency {currency} not found in API response")
        else:
            print(f"API request failed with status {response.status_code}: {response.text}")
            
    except Exception as e:
        print(f"Error fetching gold price from API: {e}")
    
    # Use demo data if API failed
    if price_per_ounce is None:
        price_per_ounce = demo_prices.get(currency, 2350.50)
        print(f"Using demo price: {price_per_ounce}")
    
    # Generate AI-powered investment advice with risk level assessment
    advice = generate_ai_investment_advice(price_per_ounce, currency, country)
    
    return {
        'price': price_per_ounce,
        'currency': currency,
        'country': country,
        'advice': advice,
        'timestamp': datetime.datetime.now().isoformat(),
        'data_source': 'API' if api_success else 'Demo'
    }

def generate_ai_investment_advice(price, currency, country):
    """Generate AI-powered investment advice using risk level assessment as a tool"""
    try:
        # First, get the risk level assessment from our analysis tool
        risk_analysis = get_risk_level_assessment(price, currency)
        risk_level = risk_analysis['risk_level']
        fallback_advice = risk_analysis['advice']
        
        # Create enhanced context for the AI advice using risk assessment as a tool
        prompt = f"""
        As a professional gold investment advisor, provide specific investment advice based on these current market conditions:
        
        MARKET DATA:
        - Current gold price: {price} {currency}
        - Country/Market: {country}
        - Currency: {currency}
        
        RISK ASSESSMENT TOOL OUTPUT:
        - Risk Level: {risk_level.upper()}
        - Price Classification: {risk_analysis['price_classification']}
        - Technical Analysis: {fallback_advice}
        
        Based on this risk assessment tool output, provide professional investment advice that:
        1. Acknowledges the {risk_level} risk level
        2. Provides specific timing recommendations
        3. Considers currency-specific factors for {currency}
        4. Offers actionable next steps
        
        Keep your advice concise (2-3 sentences), professional, and specific to the {risk_level} risk scenario.
        """
        
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "You are a professional gold investment advisor. Use the risk assessment tool output to provide specific, actionable advice. Build upon the technical analysis provided."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.7
        )
        
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        print(f"Error generating AI advice: {e}")
        # Fallback to rule-based advice if AI fails
        return generate_fallback_advice(price, currency)

def get_risk_level_assessment(price, currency):
    """Risk assessment tool that determines price level and provides technical analysis"""
    
    # Currency-specific price thresholds (approximate)
    thresholds = {
        'USD': {'low': 2000, 'moderate': 2300, 'high': 2500},
        'EUR': {'low': 1850, 'moderate': 2150, 'high': 2350},
        'GBP': {'low': 1600, 'moderate': 1850, 'high': 2100},
        'JPY': {'low': 300000, 'moderate': 340000, 'high': 380000},
        'CAD': {'low': 2700, 'moderate': 3100, 'high': 3500},
        'AUD': {'low': 3000, 'moderate': 3500, 'high': 4000},
        'INR': {'low': 160000, 'moderate': 190000, 'high': 220000},
        'CNY': {'low': 14000, 'moderate': 16500, 'high': 19000},
        'SAR': {'low': 7500, 'moderate': 8500, 'high': 9500},
        'AED': {'low': 7300, 'moderate': 8300, 'high': 9300},
        'EGP': {'low': 95000, 'moderate': 110000, 'high': 125000}
    }
    
    # Get thresholds for currency or use USD as default
    thresh = thresholds.get(currency, thresholds['USD'])
    
    if price < thresh['low']:
        return {
            'risk_level': 'low',
            'price_classification': 'Undervalued',
            'advice': f"Excellent buying opportunity! Gold is undervalued at {price} {currency}. Consider accumulating positions while prices are low."
        }
    elif price < thresh['moderate']:
        return {
            'risk_level': 'moderate', 
            'price_classification': 'Fair Value',
            'advice': f"Good entry point at {price} {currency}. Moderate pricing with growth potential. Consider dollar-cost averaging for this market."
        }
    elif price < thresh['high']:
        return {
            'risk_level': 'moderate-high',
            'price_classification': 'Fairly Valued',
            'advice': f"Fair pricing at {price} {currency}. Market is fairly valued. Consider smaller purchases or wait for pullbacks."
        }
    else:
        return {
            'risk_level': 'high',
            'price_classification': 'Premium/Overvalued',
            'advice': f"Premium pricing at {price} {currency}. Consider waiting for market corrections or focus on smaller strategic purchases."
        }

def generate_fallback_advice(price, currency):
    """Enhanced fallback advice with currency-specific considerations"""
    
    # Currency-specific price thresholds (approximate)
    thresholds = {
        'USD': {'low': 2000, 'moderate': 2300, 'high': 2500},
        'EUR': {'low': 1850, 'moderate': 2150, 'high': 2350},
        'GBP': {'low': 1600, 'moderate': 1850, 'high': 2100},
        'JPY': {'low': 300000, 'moderate': 340000, 'high': 380000},
        'CAD': {'low': 2700, 'moderate': 3100, 'high': 3500},
        'AUD': {'low': 3000, 'moderate': 3500, 'high': 4000},
        'INR': {'low': 160000, 'moderate': 190000, 'high': 220000},
        'CNY': {'low': 14000, 'moderate': 16500, 'high': 19000},
        'SAR': {'low': 7500, 'moderate': 8500, 'high': 9500},
        'AED': {'low': 7300, 'moderate': 8300, 'high': 9300},
        'EGP': {'low': 95000, 'moderate': 110000, 'high': 125000}
    }
    
    # Get thresholds for currency or use USD as default
    thresh = thresholds.get(currency, thresholds['USD'])
    
    if price < thresh['low']:
        return f"Excellent buying opportunity! Gold is undervalued at {price} {currency}. Consider accumulating positions while prices are low."
    elif price < thresh['moderate']:
        return f"Good entry point at {price} {currency}. Moderate pricing with growth potential. Consider dollar-cost averaging for this market."
    elif price < thresh['high']:
        return f"Fair pricing at {price} {currency}. Market is fairly valued. Consider smaller purchases or wait for pullbacks."
    else:
        return f"Premium pricing at {price} {currency}. Consider waiting for market corrections or focus on smaller strategic purchases."


In [ ]:
print("Testing AI-Enhanced Gold Price Function with Risk Assessment Tool:")
print("=" * 70)

# Test USA (likely high risk due to current high prices)
test_result = get_gold_price("USA")
print(f"\n🇺🇸 USA Test:")
print(f"Price: {test_result['price']} {test_result['currency']}")
print(f"Data Source: {test_result.get('data_source', 'Unknown')}")
print(f"AI Advice: {test_result['advice']}")

# Test a few more currencies to show different risk levels
print(f"\n{'='*50}")
print("Testing Different Currencies with AI Risk Assessment:")

test_currencies = ["USA", "UK", "EUR", "JPY"]
for currency in test_currencies:
    result = get_gold_price(currency)
    
    # Also show the risk assessment tool output
    risk_assessment = get_risk_level_assessment(result['price'], result['currency'])
    
    print(f"\n🌍 {currency}:")
    print(f"  Price: {result['price']} {result['currency']}")
    print(f"  Risk Level: {risk_assessment['risk_level'].upper()}")
    print(f"  Classification: {risk_assessment['price_classification']}")
    print(f"  AI Advice: {result['advice']}")
    print(f"  Data Source: {result.get('data_source', 'Unknown')}")


In [ ]:
# Fake Gold Buying Simulation
def simulate_gold_purchase(ounces, country="USD"):
    """Simulate buying gold and write to file"""
    print(f"Tool simulate_gold_purchase called for {ounces} ounces in {country}")
    
    try:
        # Get current price
        price_data = get_gold_price(country)
        
        if 'error' in price_data:
            return {'error': price_data['error']}
        
        # Ensure we have valid price data
        if 'price' not in price_data or price_data['price'] <= 0:
            return {'error': 'Invalid price data received'}
        
        price_per_ounce = float(price_data['price'])
        currency = price_data['currency']
        ounces_float = float(ounces)
        
        # Calculate total cost
        total_cost = round(price_per_ounce * ounces_float, 2)
        
        # Create detailed purchase record
        purchase_record = {
            'purchase_date': datetime.datetime.now().isoformat(),
            'ounces': ounces_float,
            'price_per_ounce': price_per_ounce,
            'total_cost': total_cost,
            'currency': currency,
            'country': country,
            'transaction_id': f"GOLD_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}",
            'price_source': price_data.get('source', 'Unknown'),
            'market_advice': price_data.get('advice', 'No advice available')
        }
        
        # Write to file
        filename = "gold_purchases.json"
        
        # Load existing purchases or create new list
        try:
            with open(filename, 'r') as f:
                purchases = json.load(f)
        except FileNotFoundError:
            purchases = []
        
        # Add new purchase
        purchases.append(purchase_record)
        
        # Save back to file
        with open(filename, 'w') as f:
            json.dump(purchases, f, indent=2)
        
        # Create detailed success message
        success_message = f"✅ Successfully purchased {ounces} ounces of gold for {total_cost:,.2f} {currency}"
        if 'note' in price_data:
            success_message += f" ({price_data['note']})"
        
        return {
            'success': True,
            'message': success_message,
            'transaction_id': purchase_record['transaction_id'],
            'total_cost': total_cost,
            'currency': currency,
            'price_per_ounce': price_per_ounce,
            'market_advice': price_data.get('advice', 'No advice available')
        }
        
    except ValueError as e:
        return {'error': f'Invalid number format: {str(e)}'}
    except Exception as e:
        return {'error': f'Purchase failed: {str(e)}'}


In [ ]:
# Test the purchase simulation
print("Testing purchase simulation...")
test_purchase = simulate_gold_purchase("2.5", "USA")
print("Purchase Result:")
print(json.dumps(test_purchase, indent=2))

In [ ]:
# Translation Agent
def translate_to_arabic(text):
    """Translate text to Arabic using OpenAI"""
    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "You are a professional translator. Translate the given text to Arabic. Maintain the meaning and tone. Only return the Arabic translation, nothing else."},
                {"role": "user", "content": text}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Translation error: {str(e)}"


In [ ]:
# Speech-to-Text Agent
def speech_to_text(audio_file):
    """Convert speech to text using OpenAI Whisper"""
    try:
        if audio_file is None:
            return "No audio file provided"
        
        with open(audio_file, "rb") as audio:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio
            )
        return transcript.text
    except Exception as e:
        return f"Speech-to-text error: {str(e)}"


In [ ]:
# Define Tools for OpenAI
gold_price_function = {
    "name": "get_gold_price",
    "description": "Get the current price of gold for a specific country/currency and investment advice. Call this when user asks about gold prices or investment timing.",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country or currency code (e.g., 'USA', 'UK', 'EUR', 'JPY')",
            },
        },
        "required": ["country"],
        "additionalProperties": False
    }
}

gold_purchase_function = {
    "name": "simulate_gold_purchase",
    "description": "Simulate purchasing gold and record the transaction. Call this when user wants to buy gold.",
    "parameters": {
        "type": "object",
        "properties": {
            "ounces": {
                "type": "string",
                "description": "Number of ounces of gold to purchase",
            },
            "country": {
                "type": "string",
                "description": "The country or currency for the purchase (e.g., 'USA', 'UK')",
            },
        },
        "required": ["ounces"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": gold_price_function},
    {"type": "function", "function": gold_purchase_function}
]


In [ ]:
# Handle Tool Calls
def handle_tool_call(message):
    """Handle tool calls from OpenAI"""
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    if function_name == "get_gold_price":
        country = arguments.get('country', 'USD')
        result = get_gold_price(country)
    elif function_name == "simulate_gold_purchase":
        ounces = arguments.get('ounces')
        country = arguments.get('country', 'USD')
        result = simulate_gold_purchase(ounces, country)
    else:
        result = {"error": "Unknown function"}
    
    response = {
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": tool_call.id
    }
    
    return response, result


In [ ]:
# Main Chat Function

def chat(history):
    """Main chat function with multi-agent capabilities"""
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    # Handle tool calls
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_response, tool_result = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]
    
    # Translate to Arabic
    arabic_translation = translate_to_arabic(reply)
    
    return history, arabic_translation


In [ ]:
# Audio Processing Function

def process_audio(audio_file, history):
    """Process audio input and add to chat"""
    if audio_file is None:
        return history, "", "No audio provided"
    
    # Convert speech to text
    transcribed_text = speech_to_text(audio_file)
    
    if transcribed_text.startswith("Speech-to-text error:"):
        return history, "", transcribed_text
    
    # Add user message to history
    history += [{"role": "user", "content": transcribed_text}]
    
    # Get response
    updated_history, arabic_translation = chat(history)
    
    return updated_history, arabic_translation, f"Transcribed: {transcribed_text}"


In [ ]:
# Advanced Gradio Interface with Multiple Panels

with gr.Blocks(title="AI Investment Estimations - Gold Assistant", theme=gr.themes.Soft()) as ui:
    gr.Markdown("# 🏆 AI Investment Estimations - Gold Assistant")
    gr.Markdown("Get real-time gold prices, investment advice, simulate purchases, and interact in multiple languages!")
    
    with gr.Row():
        # Left Panel - English Chat
        with gr.Column(scale=2):
            gr.Markdown("### 💬 English Chat")
            chatbot = gr.Chatbot(height=400, type="messages", label="Investment Assistant")
            
            with gr.Row():
                text_input = gr.Textbox(
                    label="Ask about gold prices, investment advice, or make a purchase",
                    placeholder="e.g., What's the gold price in USA? Should I invest now? Buy 2 ounces of gold"
                )
            
            with gr.Row():
                send_btn = gr.Button("Send Message", variant="primary")
                clear_btn = gr.Button("Clear Chat")
            
            # Audio Input
            gr.Markdown("### 🎤 Voice Input")
            audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak your question")
            audio_status = gr.Textbox(label="Audio Status", interactive=False)
        
        # Right Panel - Arabic Translation
        with gr.Column(scale=1):
            gr.Markdown("### 🌐 Arabic Translation")
            gr.Markdown("الترجمة العربية")
            arabic_output = gr.Textbox(
                label="Arabic Response",
                lines=15,
                interactive=False,
                text_align="right",
                rtl=True
            )
    
    # Bottom Panel - Purchase History
    with gr.Row():
        gr.Markdown("### 📊 Recent Features")
        
    with gr.Row():
        with gr.Column():
            gr.Markdown("**Supported Countries/Currencies:**")
            gr.Markdown("USA (USD), UK (GBP), Europe (EUR), Japan (JPY), Canada (CAD), Australia (AUD), India (INR), China (CNY), Saudi Arabia (SAR), UAE (AED), Egypt (EGP)")
        
        with gr.Column():
            gr.Markdown("**Features:**")
            gr.Markdown("• Real-time gold prices\\n• Investment timing advice\\n• Simulated gold purchasing\\n• Arabic translation\\n• Voice input support")
    
    # Event Handlers
    def handle_text_input(message, history):
        if not message.strip():
            return history, "", ""
        
        history += [{"role": "user", "content": message}]
        updated_history, arabic_translation = chat(history)
        return updated_history, "", arabic_translation
    
    def clear_chat():
        return [], "", ""
    
    # Connect events
    text_input.submit(
        handle_text_input,
        inputs=[text_input, chatbot],
        outputs=[chatbot, text_input, arabic_output]
    )
    
    send_btn.click(
        handle_text_input,
        inputs=[text_input, chatbot],
        outputs=[chatbot, text_input, arabic_output]
    )
    
    clear_btn.click(
        clear_chat,
        outputs=[chatbot, text_input, arabic_output]
    )
    
    audio_input.change(
        process_audio,
        inputs=[audio_input, chatbot],
        outputs=[chatbot, arabic_output, audio_status]
    )

# Launch the interface
ui.launch(inbrowser=True, share=False)


In [ ]:
# View purchase history
def view_purchase_history():
    """View all gold purchases from the JSON file"""
    try:
        with open('gold_purchases.json', 'r') as f:
            purchases = json.load(f)
        
        print("Gold Purchase History:")
        print("=" * 50)
        
        total_ounces = 0
        total_spent = {}
        
        for purchase in purchases:
            print(f"Date: {purchase['purchase_date'][:19]}")
            print(f"Ounces: {purchase['ounces']}")
            print(f"Price per ounce: {purchase['price_per_ounce']} {purchase['currency']}")
            print(f"Total cost: {purchase['total_cost']} {purchase['currency']}")
            print(f"Transaction ID: {purchase['transaction_id']}")
            print("-" * 30)
            
            total_ounces += purchase['ounces']
            currency = purchase['currency']
            if currency not in total_spent:
                total_spent[currency] = 0
            total_spent[currency] += purchase['total_cost']
        
        print(f"\nSummary:")
        print(f"Total ounces purchased: {total_ounces}")
        print(f"Total spent by currency:")
        for currency, amount in total_spent.items():
            print(f"  {currency}: {amount}")
            
    except FileNotFoundError:
        print("No purchase history found. Make your first purchase to start tracking!")
    except Exception as e:
        print(f"Error reading purchase history: {e}")

# Run this to check your purchase history
view_purchase_history()


## API Setup Instructions

### For Gold Prices API:
1. Visit https://metalpriceapi.com/
2. Sign up for a free account
3. Get your API key
4. Add `METAL_PRICE_API_KEY=your_api_key` to your .env file

### The system includes:
- **Gold Price Agent**: Fetches real-time prices with investment advice
- **Purchase Simulation Agent**: Records fake gold purchases to JSON file
- **Translation Agent**: Translates all responses to Arabic
- **Speech-to-Text Agent**: Converts voice input to text using Whisper

### Multi-Panel Interface:
- Left: English chat with text and voice input
- Right: Arabic translations
- Bottom: Feature descriptions and supported currencies

### Supported Countries/Currencies:
USA (USD), UK (GBP), Europe (EUR), Japan (JPY), Canada (CAD), Australia (AUD), India (INR), China (CNY), Saudi Arabia (SAR), UAE (AED), Egypt (EGP)
